# Tidying and Transforming Data

## (1) Import Data
Create a .CSV file that includes all of the information shown above. You are required to use a “wide” structure similar to how the information appears above, so that you can practice both data tidying and the data wrangling transformations described in Chapter 8 of the “Python for Data Analysis” textbook. Load your .CSV file into you GitHub repository.

In [427]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [428]:
# import a CSV file that created
Numbersense_df = pd.read_csv("https://raw.githubusercontent.com/Johnnydaszhu/DAV-5400/master/Numbersense.csv")
Numbersense_df

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497,221,212,503,1841
1,NaN,delayed,62,12,20,102,305
2,AM WEST,on time,694,4840,383,320,201
3,NaN,delayed,117,415,65,129,61


## (2) Tidying Data
Read your .CSV file from your GitHub repository into a Jupyter Notebook, and use your knowledge of combining and reshaping data in Pandas to tidy and transform your data. To get started, think about how you would want the data to appear if it were converted to “long” format, e.g., how would you define a “single observation” for the data shown in the graphic?; How many key values are associated with each data value?; How many columns should your long format structure contain based on the information provided in the graphic shown above?; What would the column headings for the long structure be?; etc. Use your answers to these questions to guide your reshaping/transformational work on the data. Your reshaping/transformational steps must include converting the above table to a “tidy” long format. Additional transformational steps (e.g., filling in missing data values, renaming columns, etc.) should be performed as needed to ensure that your data is, in fact, “tidy”.

In [429]:
# Convert the table to Long format using Month & Category as the keys and 'Los Angeles', 'Phoenix', 'San Diego','San Francisco','Seattle' as the data values
long_df = Numbersense_df.melt(id_vars = ['Unnamed: 0', 'Unnamed: 1'], value_vars = ['Los Angeles', 'Phoenix', 'San Diego','San Francisco','Seattle'])
long_df.head()

,Unnamed: 0,Unnamed: 1,variable,value
0,ALASKA,on time,Los Angeles,497
1,NaN,delayed,Los Angeles,62
2,AM WEST,on time,Los Angeles,694
3,NaN,delayed,Los Angeles,117
4,ALASKA,on time,Phoenix,221


In [430]:
# fill the NaN items in the 'Unnamed: 0' column
long_df['Unnamed: 0'].fillna(method='ffill', inplace = True)
long_df.head()

,Unnamed: 0,Unnamed: 1,variable,value
0,ALASKA,on time,Los Angeles,497
1,ALASKA,delayed,Los Angeles,62
2,AM WEST,on time,Los Angeles,694
3,AM WEST,delayed,Los Angeles,117
4,ALASKA,on time,Phoenix,221


In [431]:
# rename the columns
long_df.columns = ['Airlines', 'Status','City','FlightNumber']
long_df

,Airlines,Status,City,FlightNumber
0,ALASKA,on time,Los Angeles,497
1,ALASKA,delayed,Los Angeles,62
2,AM WEST,on time,Los Angeles,694
3,AM WEST,delayed,Los Angeles,117
4,ALASKA,on time,Phoenix,221
5,ALASKA,delayed,Phoenix,12
6,AM WEST,on time,Phoenix,4840
7,AM WEST,delayed,Phoenix,415
8,ALASKA,on time,San Diego,212
9,ALASKA,delayed,San Diego,20


Now we get a tidy data

## (3) Data Analysis
Using your reshaped/transformed data, perform analysis to compare the arrival delays for the two airlines. Some questions you might choose to answer: For each city, which airline had the best on time performance?; Which airline had the best overall on time performance?, etc.

In [432]:
# find the average On time rate for each airline
total = long_df.groupby(['Airlines']).sum()
on_time = long_df.loc[long_df['Status'] == 'on time']
g = on_time.groupby(['Airlines']).sum()/(total.groupby(['Airlines']).sum())
g.columns = ['Average_OnTime_Rate']
g

,Average_OnTime_Rate
Airlines,
ALASKA,0.867285
AM WEST,0.891073


### Insight 1: AM WEST has better overall on time performance

In [433]:
# find the average On time rate for each city
total_c = long_df.groupby(['City']).sum()
on_time_c = long_df.loc[long_df['Status'] == 'on time']
c = on_time_c.groupby(['City']).sum()/(total_c.groupby(['City']).sum())
c.columns = ['Average_OnTime_Rate']
c.sort_values(by=['Average_OnTime_Rate'],ascending=False)


,Average_OnTime_Rate
City,
Phoenix,0.922194
San Diego,0.875000
Los Angeles,0.869343
Seattle,0.848007
San Francisco,0.780835


### Insight 2: Phoenix has the best Average On Time Rate while San Francisco has the worst

In [434]:
# add the percentage column of each flight
new_df = on_time.groupby(['Airlines','City']).sum()
new_df.columns = ['OnTimeFlightNumber']
new_df['Total'] = long_df.groupby(['Airlines','City']).sum()
new_df['Percentage'] = new_df['OnTimeFlightNumber'] / new_df['Total']
new_df

OnTimeFlightNumber  Total  Percentage
Airlines City                                                
ALASKA   Los Angeles                   497    559    0.889088
         Phoenix                       221    233    0.948498
         San Diego                     212    232    0.913793
         San Francisco                 503    605    0.831405
         Seattle                      1841   2146    0.857875
AM WEST  Los Angeles                   694    811    0.855734
         Phoenix                      4840   5255    0.921028
         San Diego                     383    448    0.854911
         San Francisco                 320    449    0.712695
         Seattle                       201    262    0.767176

In [435]:
# merge two dataframe and add a column of comparison about each airline average on time rate
in_df= pd.merge(new_df, g, left_on='Airlines', right_index=True)
in_df['Better in Airline?'] = in_df['Percentage'] >= in_df['Average_OnTime_Rate']
in_df.drop(columns=['Average_OnTime_Rate'], inplace = True)
in_df

OnTimeFlightNumber  Total  Percentage  \
        City                                                   
ALASKA  Los Angeles                   497    559    0.889088   
        Phoenix                       221    233    0.948498   
        San Diego                     212    232    0.913793   
        San Francisco                 503    605    0.831405   
        Seattle                      1841   2146    0.857875   
AM WEST Los Angeles                   694    811    0.855734   
        Phoenix                      4840   5255    0.921028   
        San Diego                     383    448    0.854911   
        San Francisco                 320    449    0.712695   
        Seattle                       201    262    0.767176   

                       Better in Airline?  
        City                               
ALASKA  Los Angeles                  True  
        Phoenix                      True  
        San Diego                    True  
        San Francisco               False  
        Seattle                     False  
AM WEST Los Angeles                 False  
        Phoenix                      True  
        San Diego                   False  
        San Francisco               False  
        Seattle                     False

### Insight 3: We could find how each airline perform among different cities

In [436]:
# merge two dataframe and add a column of comarison to another airline in each city
out_df= pd.merge(in_df, c, left_on='City', right_index=True)
out_df['Better than another Airline?'] = out_df['Percentage'] >= out_df['Average_OnTime_Rate']
out_df.drop(columns=['Average_OnTime_Rate'], inplace = True)
out_df.sort_values(by=['Percentage'],ascending=False)

OnTimeFlightNumber  Total  Percentage  \
ALASKA  Phoenix                       221    233    0.948498   
AM WEST Phoenix                      4840   5255    0.921028   
ALASKA  San Diego                     212    232    0.913793   
        Los Angeles                   497    559    0.889088   
        Seattle                      1841   2146    0.857875   
AM WEST Los Angeles                   694    811    0.855734   
        San Diego                     383    448    0.854911   
ALASKA  San Francisco                 503    605    0.831405   
AM WEST Seattle                       201    262    0.767176   
        San Francisco                 320    449    0.712695   

                       Better in Airline?  Better than another Airline?  
ALASKA  Phoenix                      True                          True  
AM WEST Phoenix                      True                         False  
ALASKA  San Diego                    True                          True  
        Los Angeles                  True                          True  
        Seattle                     False                          True  
AM WEST Los Angeles                 False                         False  
        San Diego                   False                         False  
ALASKA  San Francisco               False                          True  
AM WEST Seattle                     False                         False  
        San Francisco               False                         False

### Insight 4: We could find which airline could perform better in each city

## (4) Wide format
Finally, given your “tidy” long format structure, consider what, if any, changes you would make to the visual presentation of the data if you were then asked to transform your “long” data back into a “wide” format: would you mimic the structure of the graphic shown above? If not, how might you transform your “long” data to “wide” format to make its “wide” presentation easier to understand and work with? Provide an example of your recommendation.

In [437]:
long_df.groupby(['Airlines','City','Status']).sum()

FlightNumber
Airlines City          Status               
ALASKA   Los Angeles   delayed            62
                       on time           497
         Phoenix       delayed            12
                       on time           221
         San Diego     delayed            20
                       on time           212
         San Francisco delayed           102
                       on time           503
         Seattle       delayed           305
                       on time          1841
AM WEST  Los Angeles   delayed           117
                       on time           694
         Phoenix       delayed           415
                       on time          4840
         San Diego     delayed            65
                       on time           383
         San Francisco delayed           129
                       on time           320
         Seattle       delayed            61
                       on time           201

For visual presentation, I would stack the data and make it a bit longer but would not long as the tidy data shown before. 
With this level of group, we could clearly match different Airlines, City and their Status.